In [ ]:
import numpy as np
import onnxruntime as ort
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx
from onnxruntime.quantization import QuantType, quantize_dynamic

import onnx


### float32→int8

In [2]:
MIN=-200 # データの最小値
MAX=412 # データの最大値

S = (255/(MAX-MIN)) # scale

_MIN = MIN*S
_MAX = MAX*S
_MIN,_MAX  # -127~128の範囲に入っていないのでzero-pointで調整

(-83.33333333333334, 171.66666666666669)

In [5]:
# zero-point
print(f"{np.abs(-128-_MIN)},{np.abs(127-_MAX)}")
print(f"{round(np.abs(-128-_MIN))},{round(np.abs(127-_MAX))}")

44.66666666666666,44.666666666666686
45,45


126.66666666666669

### onnxruntime.quantization.preprocess

In [24]:

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.linear = nn.Linear(10, 1)  # 10入力1出力の全結合層

    def forward(self, x):
        return self.linear(x)
# モデルのインスタンス化
model = SimpleNN()

# 推論を行うためのサンプルテンソル（ダミーデータ）を作成
# ここでは、バッチサイズを2とする（2つのサンプルがあると仮定）
sample_tensor = torch.randn(1, 10)  # 2x10のテンソル

# モデルを評価モードに設定
model.eval()

# 推論実行
# with torch.no_grad():  # 勾配計算を無効化
#     output = model(sample_tensor)
# print(output),sample_tensor

model.eval()

output_onnx = 'simple_one.onnx'
torch.onnx.export(model, sample_tensor, output_onnx, verbose=True, input_names=['input'], output_names=['output'])

In [25]:
!python -m onnxruntime.quantization.preprocess --input simple_one.onnx --output simple_one-infer.onnx

In [11]:
model = models.resnet18(pretrained=False)
model.eval()  # モデルを評価モードに設定
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, dummy_input, "resnet18.onnx", verbose=True, input_names=['input'], output_names=['output'])

In [12]:
!python -m onnxruntime.quantization.preprocess --input resnet18.onnx --output resnet18-infer.onnx
